In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
%%time
df = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df.head()

CPU times: user 2.27 s, sys: 184 ms, total: 2.45 s
Wall time: 2.46 s


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


# Preporcess data

In [4]:
from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
from tqdm import tqdm
import numpy as np
import datetime
import itertools

mapping_ord1 = {'Unknown': 0, 'Novice': 1, 'Expert': 2, 'Contributor': 3, 'Master': 4, 'Grandmaster': 5}
mapping_ord2 = {'Unknown': 0, 'Freezing': 1, 'Cold': 2, 'Warm': 3, 'Hot': 4, 'Boiling Hot': 5, 'Lava Hot': 6}
mapping_ord3 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_3'].fillna("0"))))])
mapping_ord4 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_4'].fillna("0"))))])
mapping_ord5 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_5'].fillna("0"))))])


def combine_cols(df, cols):
    comb_cols = []
    for i, j in tqdm(itertools.combinations(cols, 2)):
        c = i + "_" + j
        df[c] = (df[i].fillna("Unknown").astype(str) + "_" + df[j].fillna("Unknown").astype(str)).values
        comb_cols.append(c)
    return df, comb_cols

def preprocess_data(df):
    df['ord_0'] = df['ord_0'].fillna(0)
    df['ord_1'] =  df['ord_1'].fillna('Unknown').map(mapping_ord1)
    df['ord_2'] =  df['ord_2'].fillna('Unknown').map(mapping_ord2)
    df['ord_3'] =  df['ord_3'].fillna('0').map(mapping_ord3)
    df['ord_4'] =  df['ord_4'].fillna('0').map(mapping_ord4)
    df['ord_5'] =  df['ord_5'].fillna('0').map(mapping_ord5)
    df['bin_3'] = df['bin_3'].fillna('U').map({"T": 1, "F": 0, "U": np.nan})
    df['bin_4'] = df['bin_4'].fillna('U').map({"Y": 1, "N": 0, "U": np.nan})
    return df

def encode(df, df_test, cols):
    cols_enc = list(map(lambda x: x + "_enc", cols))
    for c in cols_enc:
        df[c] = np.nan
        df_test[c] = np.nan
        
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    for train_idx, oof_idx in tqdm(skf.split(df, df["target"])):
        enc = ce.TargetEncoder(cols = cols, smoothing=0.3)
        enc.fit(df.loc[train_idx, cols], df.loc[train_idx, "target"])
        df.loc[oof_idx, cols_enc] = enc.transform(df.loc[oof_idx, cols]).values
    
    enc = ce.TargetEncoder(cols = cols, smoothing=0.3)
    enc.fit(df[cols], df["target"])
    df_test[cols_enc] = enc.transform(df_test[cols])
    return df, df_test, cols_enc
    
df = preprocess_data(df)
df_test = preprocess_data(df_test)

binary = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
ordinal  = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
low_card = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']
high_card = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
date = ['day', 'month']

df, df_test, cat_enc =  encode(df, df_test, high_card + low_card + date)
features = binary + ordinal + cat_enc

5it [00:15,  3.01s/it]


In [5]:
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_7_enc,nom_8_enc,nom_9_enc,nom_0_enc,nom_1_enc,nom_2_enc,nom_3_enc,nom_4_enc,day_enc,month_enc
0,0,0.0,0.0,0.0,0.0,0.0,Red,Trapezoid,Hamster,Russia,...,0.098296,0.129236,0.177914,0.183158,0.225096,0.167919,0.219491,0.207147,0.200424,0.146475
1,1,1.0,1.0,0.0,0.0,1.0,Red,Star,Axolotl,NaN,...,0.128414,0.191517,0.155502,0.183176,0.130233,0.200552,0.185771,0.179593,0.212892,0.208686
2,2,0.0,1.0,0.0,0.0,0.0,Red,NaN,Hamster,Canada,...,0.266392,0.169176,0.188439,0.183158,0.181900,0.167919,0.159677,0.207147,0.164081,0.213645
3,3,NaN,0.0,0.0,0.0,0.0,Red,Circle,Hamster,Finland,...,0.139367,0.222915,0.271186,0.182968,0.179723,0.168632,0.178692,0.179306,0.163514,0.146270
4,4,0.0,NaN,0.0,1.0,0.0,Red,Triangle,Hamster,Costa Rica,...,0.195893,0.191633,0.261851,0.183176,0.157501,0.167905,0.202839,0.188604,0.163069,0.224929


In [6]:
df.shape

(600000, 37)

# Learn model

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

In [8]:
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
import category_encoders as ce
 
params = {
        'clf__objective': 'binary',
        'clf__boosting_type': 'gbdt', 
        'clf__metric': 'auc',
        'clf__learning_rate': 0.01,
        "clf__bagging_freq": 5,
        "clf__bagging_fraction": 0.8,
        "clf__min_data_in_leaf": 30,
        "clf__min_sum_hessian_in_leaf": 5,
        "clf__use_two_round_loading": False,
        "clf__feature_fraction": 0.8,
        'clf__verbose': 1,
        "clf__lambda_l1": 0.1,
        "clf__n_estimators": 5000,
        "clf__max_depth": 3,
    }

                        
pipeline = Pipeline(steps=[('sel', ColumnSelector(features)), 
                           ('clf', LGBMClassifier())]).set_params(**params)

In [9]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt.pyll.base import scope
from hyperopt import space_eval
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


def make_objective(pipeline, df):
    
    def objective(params):
                
        pipeline.set_params(**params)
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        scores = []
        for train_idx, val_idx in tqdm(skf.split(df, df["target"])):
            
            df_train, df_val = df.loc[train_idx, :], df.loc[val_idx, :]
            transformers = Pipeline(pipeline.steps[:-1]).fit(df_train, df_train['target'])
            X_val, y_val = transformers.transform(df_val), df_val['target']
            
            fit_args = {
                "clf__early_stopping_rounds": 1000, 
                "clf__eval_set": (X_val, y_val),
                "clf__verbose": 500,
                "clf__eval_metric": 'auc',
            }
            
            pipeline.fit(df_train, df_train['target'], **fit_args)
            prob = pipeline.predict_proba(df_val)[:, 1]
            scores.append(roc_auc_score(y_val, prob))
            
        score = np.mean(scores)
        print("AUC {:.5f}+-{:.5f}.\n".format(np.mean(scores), np.std(scores)) + 
              "Params: \n" + 
              "\n".join(["{}: {:.3f}".format(k, params[k]) for k in params]))
        return 1 - score
    
    return objective

space = {
        "clf__learning_rate": hp.loguniform("clf__learning_rate", np.log(0.005), np.log(0.5)),
        'clf__lambda_l1': hp.loguniform('clf__lambda_l1', np.log(0.005), np.log(0.5)),
        "clf__num_leaves": hp.choice("clf__num_leaves", np.linspace(8, 128, 50, dtype=int)),
        "clf__feature_fraction": hp.quniform("clf__feature_fraction", 0.5, 1.0, 0.1),
        "clf__bagging_fraction": hp.quniform("clf__bagging_fraction", 0.5, 1.0, 0.1),
        "clf__min_child_weight": hp.uniform('clf__min_child_weight', 0.5, 10),
        'clf__colsample_bytree': hp.uniform('clf__colsample_bytree', 0.3, 1.0),
    }

best = fmin(fn=make_objective(pipeline, df),
            space=space,
            algo=tpe.suggest,
            max_evals=10)

best_params = space_eval(space, best)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

0it [00:00, ?it/s]



Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.775119                         
[1000]	valid_0's auc: 0.783153                        
[1500]	valid_0's auc: 0.785433                        
[2000]	valid_0's auc: 0.786188                        
[2500]	valid_0's auc: 0.786422                        
[3000]	valid_0's auc: 0.78645                         
[3500]	valid_0's auc: 0.786444                        
Early stopping, best iteration is:                    
[2853]	valid_0's auc: 0.78648
  0%|          | 0/10 [00:47<?, ?trial/s, best loss=?]

1it [00:49, 49.55s/it]



Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.777409                         
[1000]	valid_0's auc: 0.785385                        
[1500]	valid_0's auc: 0.787817                        
[2000]	valid_0's auc: 0.788578                        
[2500]	valid_0's auc: 0.788816                        
[3000]	valid_0's auc: 0.788879                        
[3500]	valid_0's auc: 0.788864                        
[4000]	valid_0's auc: 0.788847                        
[4500]	valid_0's auc: 0.788772                        
Early stopping, best iteration is:                    
[3590]	valid_0's auc: 0.788897
  0%|          | 0/10 [01:48<?, ?trial/s, best loss=?]

...

Training until validation scores don't improve for 1000 rounds                    
[500]	valid_0's auc: 0.782431                                                     
[1000]	valid_0's auc: 0.780208                                                    
Early stopping, best iteration is:                                                
[126]	valid_0's auc: 0.783822
 90%|█████████ | 9/10 [25:54<02:51, 171.95s/trial, best loss: 0.21236755207045555]

4it [01:03, 15.75s/it]



Training until validation scores don't improve for 1000 rounds                    
[500]	valid_0's auc: 0.783432                                                     
[1000]	valid_0's auc: 0.781335                                                    
Early stopping, best iteration is:                                                
[185]	valid_0's auc: 0.784669
 90%|█████████ | 9/10 [26:10<02:51, 171.95s/trial, best loss: 0.21236755207045555]

5it [01:19, 15.82s/it]

5it [01:19, 15.83s/it]


AUC 0.78538+-0.00124.                                                             
Params: 
clf__bagging_fraction: 0.800
clf__colsample_bytree: 0.971
clf__feature_fraction: 0.500
clf__lambda_l1: 0.360
clf__learning_rate: 0.296
clf__min_child_weight: 7.139
clf__num_leaves: 49.000
100%|██████████| 10/10 [26:10<00:00, 157.05s/trial, best loss: 0.21236755207045555]


In [10]:
print("Best params: \n" + "\n".join(["{}: {:.3f}".format(k, best_params[k]) for k in best_params]))

Best params: 
clf__bagging_fraction: 0.600
clf__colsample_bytree: 0.418
clf__feature_fraction: 0.900
clf__lambda_l1: 0.016
clf__learning_rate: 0.009
clf__min_child_weight: 7.577
clf__num_leaves: 79.000


In [11]:
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
pipeline.set_params(**best_params)
df_test['target'] = 0

for train_idx, val_idx in tqdm(skf.split(df, df['target'])):
    df_train = df.loc[train_idx, :]
    df_val = df.loc[val_idx, :]
    X_val, y_val = Pipeline(pipeline.steps[:-1]).fit(df_train, df_train['target'])\
                                                    .transform(df_val), df_val['target']
    fit_args = {
        "clf__early_stopping_rounds": 1000, 
        "clf__eval_set": (X_val, y_val),
        "clf__verbose": 500,
        "clf__eval_metric": 'auc'
    }
        
    pipeline.fit(df_train, df_train['target'], **fit_args)
    prob = pipeline.predict_proba(df_val)[:, 1]
    score = roc_auc_score(y_val, prob)
    
    df_test['target'] += pipeline.predict_proba(df_test)[:, 1]


0it [00:00, ?it/s]

Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.763444
[1000]	valid_0's auc: 0.777578
[1500]	valid_0's auc: 0.782176
[2000]	valid_0's auc: 0.784353
[2500]	valid_0's auc: 0.785476
[3000]	valid_0's auc: 0.786085
[3500]	valid_0's auc: 0.786385
[4000]	valid_0's auc: 0.786541
[4500]	valid_0's auc: 0.78659
[5000]	valid_0's auc: 0.786578
Did not meet early stopping. Best iteration is:
[4644]	valid_0's auc: 0.7866


1it [01:13, 73.65s/it]

Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.765766
[1000]	valid_0's auc: 0.779393
[1500]	valid_0's auc: 0.784389
[2000]	valid_0's auc: 0.786697
[2500]	valid_0's auc: 0.787944
[3000]	valid_0's auc: 0.788484
[3500]	valid_0's auc: 0.788771
[4000]	valid_0's auc: 0.788915
[4500]	valid_0's auc: 0.789019
[5000]	valid_0's auc: 0.789048
Did not meet early stopping. Best iteration is:
[4984]	valid_0's auc: 0.789056


2it [02:31, 74.78s/it]

...

3it [04:08, 81.67s/it]

Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.764825
[1000]	valid_0's auc: 0.778062
[1500]	valid_0's auc: 0.782507
[2000]	valid_0's auc: 0.784631
[2500]	valid_0's auc: 0.785717
[3000]	valid_0's auc: 0.786186
[3500]	valid_0's auc: 0.786446
[4000]	valid_0's auc: 0.78656
[4500]	valid_0's auc: 0.786621
[5000]	valid_0's auc: 0.786625
Did not meet early stopping. Best iteration is:
[4670]	valid_0's auc: 0.786632


4it [05:47, 86.74s/it]

Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.762881
[1000]	valid_0's auc: 0.77702
[1500]	valid_0's auc: 0.781843
[2000]	valid_0's auc: 0.784202
[2500]	valid_0's auc: 0.78543
[3000]	valid_0's auc: 0.786058
[3500]	valid_0's auc: 0.786356
[4000]	valid_0's auc: 0.786497
[4500]	valid_0's auc: 0.78656
[5000]	valid_0's auc: 0.786558
Did not meet early stopping. Best iteration is:
[4790]	valid_0's auc: 0.78659


5it [07:31, 90.38s/it]


In [12]:
df_test['target'] /= folds
df_test[['id', 'target']].to_csv('../data/submission_lgbm.csv', index=False)
!head -n 5 ../data/submission_lgbm.csv

id,target
600000,0.10387206060714602
600001,0.2675943182020537
600002,0.1818472226517094
600003,0.10811305099184843
